# Recommending Music to train to using an existing Spotify playlist

*Code by Sean Brockway*

### Necessary imports for Spotify and MongoDB Connections

In [2]:
import requests
import json
import pymongo
import spotipy
import spotipy.util as util 
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from spotipy.oauth2 import SpotifyClientCredentials
from pymongo import MongoClient
import urllib.parse

### Setting up connections for MongoDB and Spotify API

In [9]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

conn = MongoClient("mongodb://localhost:27017/")
db = conn['gym-music-database']
test_playlist_col = db['Sam Playlist 1']
playlist_songs = test_playlist_col.find({})

**Verifying our data is correct, this is an example of a song in our playlist**

In [10]:
print(playlist_songs[0])

{'_id': ObjectId('5eaab3e9cc090b818b3fa6f6'), 'track': "Camille - Bande originale du film 'Le mépris'", 'track_id': '7hNqtgKzaUGWdwbar8Ugl1', 'artist_name': ['Georges Delerue'], 'artist_id': ['5C2ILE4DWY1vzn1NPxJOag'], 'genres': ['classic soundtrack', 'deep soundtrack', 'french soundtrack', 'soundtrack'], 'danceability': 0.187, 'energy': 0.268, 'key': 6, 'loudness': -13.398, 'mode': 0, 'speechiness': 0.0366, 'acousticness': 0.991, 'instrumentalness': 0.902, 'liveness': 0.116, 'valence': 0.0367, 'tempo': 71.533, 'duration_ms': 151147, 'time_signature': 3}


**Next we need to make a list of every song's genres**

In [14]:
genre_list = []
for song in playlist_songs:
    genre_list.extend(song['genres'])
genre_list

['classic soundtrack',
 'deep soundtrack',
 'french soundtrack',
 'soundtrack',
 'compositional ambient',
 'icelandic classical',
 'icelandic experimental',
 'compositional ambient',
 'icelandic classical',
 'icelandic experimental',
 'british soundtrack',
 'epicore',
 'scorecore',
 'soundtrack',
 'compositional ambient',
 'focus',
 'post-minimalism',
 'soundtrack',
 'compositional ambient',
 'neo-classical',
 'british soundtrack',
 'ballet class',
 'ballet class',
 'ballet class',
 'chamber orchestra',
 'soundtrack',
 'bow pop',
 'compositional ambient',
 'dream pop',
 'ethereal wave',
 'icelandic pop',
 'icelandic rock',
 'melancholia',
 'nordic post-rock',
 'post-rock',
 'scorecore',
 'soundtrack',
 'video game music',
 'scorecore',
 'soundtrack',
 'video game music',
 'scorecore',
 'soundtrack',
 'video game music',
 'scorecore',
 'soundtrack',
 'video game music',
 'downtempo',
 'electronica',
 'ninja',
 'nu jazz',
 'british post-rock',
 'cosmic post-rock',
 'post-rock',
 'ambient

**Next we need to find the most common occuring genres to base our recommendations on**

In [5]:
from collections import Counter
most_common_genres = [genre for genre, genre_count in Counter(genre_list).most_common(8)]
print(most_common_genres)

['compositional ambient', 'drone', 'ambient', 'focus', 'soundtrack', 'drift', 'post-rock', 'warm drone']


To get ideas for songs to recommend, the search end point and recommendations end point in the API can be used. For the search end point, spaces will not work, so it is best to use the "replace" method to change spaces to "%20"

The recommendations end point in the Spotify API can also be used to generate a list of song recommendations, this does not require any text to be replaced. 

In [6]:
search_genres = []
for genre in most_common_genres:
    search = genre.replace(" ", "%20")
    search_genres.append(search)
print(search_genres)

['compositional%20ambient', 'drone', 'ambient', 'focus', 'soundtrack', 'drift', 'post-rock', 'warm%20drone']


#### Now find recommendations using the 5 most common genres. 
It is worth noting that the recommendations end point has lots of different paramaters and kwargs, and bespoke recommendations can be made using the detailed spotify audio features. For now though, we will just use genre.

Firstly we need to get all the available genre seeds used to generate recommendations

In [7]:
seeds = sp.recommendation_genre_seeds()
print(seeds['genres'])

['acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'bossanova', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic', 'emo', 'folk', 'forro', 'french', 'funk', 'garage', 'german', 'gospel', 'goth', 'grindcore', 'groove', 'grunge', 'guitar', 'happy', 'hard-rock', 'hardcore', 'hardstyle', 'heavy-metal', 'hip-hop', 'holidays', 'honky-tonk', 'house', 'idm', 'indian', 'indie', 'indie-pop', 'industrial', 'iranian', 'j-dance', 'j-idol', 'j-pop', 'j-rock', 'jazz', 'k-pop', 'kids', 'latin', 'latino', 'malay', 'mandopop', 'metal', 'metal-misc', 'metalcore', 'minimal-techno', 'movies', 'mpb', 'new-age', 'new-release', 'opera', 'pagode', 'party', 'philippines-opm', 'piano', 'pop', 'pop-film', 'post-dubstep', 'power-po

Now we find the genres in our playlist that can be used as a genre seed

In [9]:
list_A = most_common_genres
list_B = seeds['genres']
jVal = "|".join(list_A)

matches = [i for i in list_B if i in jVal ]
truncated_matches = matches[:-2]
print(matches)

['ambient', 'rock']


Since there are only two matching genres and the maximum seeds are 5, we can add 3 artists as a recommendation seeds. Lets find the most common artists by firstly making a list of the artists in the playlist.

In [17]:
playlist_songs = test_playlist_col.find({})
artist_list = []
for song in playlist_songs:
    artist_list.extend(song['artist_id'])
artist_list

['5C2ILE4DWY1vzn1NPxJOag',
 '5AVGx9Dhw7OLeJ5at3Y19f',
 '5AVGx9Dhw7OLeJ5at3Y19f',
 '2vuXYcTqreOLLnlKtMV5UG',
 '2VZNmg4vCnew4Pavo8zDdW',
 '557AjoqV9wVpa8vO2k4wuJ',
 '4d6eKLLJbe2ZeiL07Du7vB',
 '6s8AUvcmf2fp0Kh7PctPnd',
 '6qzi6mPoJU4cBlO76U1Il1',
 '33TpuAICGC2gekoZZDDZhE',
 '5RLsx9QFxt7RO5jTOxzWzl',
 '6tRO6Fmf0ux4iozujurtlx',
 '5RLsx9QFxt7RO5jTOxzWzl',
 '5RLsx9QFxt7RO5jTOxzWzl',
 '2IS9rTtWwEfUEY4u3mecIT',
 '6UUrUCIZtQeOf8tC0WuzRy',
 '77yY2QmM6bYvjJ3y5L2R0v',
 '77yY2QmM6bYvjJ3y5L2R0v',
 '77yY2QmM6bYvjJ3y5L2R0v',
 '77yY2QmM6bYvjJ3y5L2R0v',
 '32ogthv0BdaSMPml02X9YB',
 '34UhPkLbtFKRq3nmfFgejG',
 '4QK3YJ6hzJdhJHE9q7kbVV',
 '1SUrVMfhMMaO87E57Bazob',
 '4QK3YJ6hzJdhJHE9q7kbVV',
 '7E3BRXV9ZbCt5lQTCXMTia',
 '3IWcM6XNOqH8UA3AQIes2B',
 '7E3BRXV9ZbCt5lQTCXMTia',
 '2fqgzLpPOMc7GMw9qcOfQT',
 '2wyAa4WC10hLoq4ye80EUd',
 '0VOR7Ie9xUSb45fzIIVJQ1',
 '1csBgT42N4pPPs1HJhxXIK',
 '3aKoUd44IrrqOa6Rzn9Ybq',
 '3ZiiIxwcdE60xHzCQ35ewj',
 '2lD1D6eEh7xQdBtnl2Ik7Y',
 '4QK3YJ6hzJdhJHE9q7kbVV',
 '1WudHeuEN3d18SXVos95mc',
 

In [18]:
most_common_artists = [artist for artist, artist_count in Counter(artist_list).most_common(3)]
print(most_common_artists)

['0grPfzk6cTnzfQpxjLDPs0', '13CMfEeq8AC6ryGOPODqE1', '4QK3YJ6hzJdhJHE9q7kbVV']


Now we have 3 artist seeds and 2 genre seeds, we can build or query parameters

In [30]:
recommended_tracks = sp.recommendations(seed_genres = matches, seed_artists = most_common_artists, limit=20)
print(recommended_tracks['tracks'])

[{'album': {'album_type': 'ALBUM', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4G1ZsxfEEztbE1VcnNInPg'}, 'href': 'https://api.spotify.com/v1/artists/4G1ZsxfEEztbE1VcnNInPg', 'id': '4G1ZsxfEEztbE1VcnNInPg', 'name': 'Chihei Hatakeyama', 'type': 'artist', 'uri': 'spotify:artist:4G1ZsxfEEztbE1VcnNInPg'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/0sBlCjVNskfvbNAHwltpIU'}, 'href': 'https://api.spotify.com/v1/albums/0sBlCjVNskfvbNAHwltpIU', 'id': '0sBlCjV

Here are the recommended tracks from Spotify, now we need to make sure there are no duplicate tracks that exist in the existing playlist, then we need to get the audio features for each track and prepare them for our classifier. 

In [31]:
for track  in recommended_tracks['tracks']:
    if any(track['id'] in songs for songs in playlist_songs):
        print('match found')
    else:
        print('no match')

no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match
no match


No matches have been found, now we can begin getting the audio features for each song and building a data frame for our model to make predictions on. 

In [32]:
dataframe_columns = ['track', 'track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
song_list = []
for track in recommended_tracks['tracks']:
    features = sp.audio_features(track['id'])
    if (features[0] is not None):
                song_record = { 
                    "track": track['name'], 
                    "track_id": track['id'],
                    "danceability" : features[0]['danceability'],
                    "energy" : features[0]['energy'],
                    "key" : features[0]['key'],
                    "loudness" : features[0]['loudness'],
                    "mode" : features[0]['mode'],
                    "speechiness" : features[0]['speechiness'],
                    "acousticness" : features[0]['acousticness'],
                    "instrumentalness" : features[0]['instrumentalness'],
                    "liveness" : features[0]['liveness'],
                    "valence" : features[0]['valence'],
                    "tempo" : features[0]['tempo'],
                    "duration_ms" : features[0]['duration_ms'],
                    "time_signature" : features[0]['time_signature']
                } 
                song_list.append(song_record)
#print(song_list)
dfRecommendations = pd.DataFrame.from_dict(song_list)
dfRecommendations

,track,track_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,White Light,4sfORtxIQnO42AJpNy8Gsq,0.2060,0.003790,2,-27.297,1,0.0470,0.987000,0.989000,0.0975,0.0717,67.815,127880,3
1,Keep,3OjWW823M5rbDuIr9mB06d,0.4760,0.766000,9,-11.686,0,0.0440,0.843000,0.967000,0.2800,0.7200,166.820,205896,3
2,Starwood Choker,2w5l9sDXytt4lu0ZYFSoIX,0.1530,0.198000,11,-19.542,1,0.0333,0.903000,0.933000,0.0977,0.0356,85.738,376956,4
3,Back to the Start,4Pdnt4VOAv26b1ILvtAoRj,0.5600,0.735000,2,-8.809,1,0.0394,0.027600,0.938000,0.1100,0.2290,108.016,287093,4
4,Regenerative Being,1Rz3x1bF09JeEj2cTWH2mg,0.0855,0.353000,9,-11.264,0,0.0394,0.543000,0.876000,0.0999,0.0619,66.260,441400,4
5,Merry,04NxrMXtaX4zjdwN12SdCF,0.3190,0.006910,0,-33.364,1,0.0411,0.986000,0.931000,0.0944,0.0375,96.772,248114,3
6,Melancholia II,2UZXQrEi0A8ySbDuyeuMt3,0.1530,0.039100,3,-32.652,1,0.0362,0.983000,0.928000,0.0774,0.0737,69.906,410987,5
7,Tears Don't Fall,5pKCDm2fw4k6D6C5Rk646C,0.2140,0.931000,7,-3.347,0,0.1430,0.000705,0.000003,0.0797,0.2330,162.115,348387,4
8,Fyrsta,1EEJBUOORu9dLZQSeQkZOP,0.2310,0.074900,0,-20.969,0,0.0351,0.989000,0.933000,0.1120,0.0735,134.312,256507,4
9,Reflector,1kAD3OQeKWZoT3oNj5bKLx,0.0663,0.036700,10,-26.219,1,0.0469,0.860000,0.927000,0.1230,0.0338,75.043,440592,4


Now we have a data frame which we can use to predict if a song is gym suitable or not. Before this happens, we need to train our model. We will do this with 10 thousands records of training data using the random forest algorthim. 

In [33]:
training_col = db['medium_test_songlist']
training_songs = training_col.find({})
dfTraining =  pd.DataFrame(list(training_songs))
dfTraining

,_id,track,track_id,artist_name,artist_id,from_playlist,genres,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,workout_suitable
0,5e7e4fa0b96e22bd0ee61d99,'Till I Collapse,4xkOaSrkexMciUUogZKVTS,"[Eminem, Nate Dogg]","[7dGJo4pcD2V6oG8kP0tJRR, 1Oa0bMld0A3u5OTYfMzp5h]","Gym Motivation : Workout Motivation, Training ...","[detroit hip hop, g funk, hip hop, rap, g funk...",0.548,0.8470,1,...,1,0.1860,0.06220,0.000000,0.0816,0.1000,171.447,297787,4,1
1,5e7e4fa1b96e22bd0ee61d9a,Numb / Encore,5sNESr6pQfIhL3krM8CtZn,"[JAY-Z, Linkin Park]","[3nFkdlSjzX9mRTtwJOzDYB, 6XyY86QOPPrYVGvF9ch6wz]","Gym Motivation : Workout Motivation, Training ...","[east coast hip hop, hip hop, pop rap, rap, al...",0.687,0.7930,2,...,1,0.1660,0.06030,0.000000,0.5820,0.7510,107.045,205733,4,1
2,5e7e4fa1b96e22bd0ee61d9b,Remember the Name (feat. Styles of Beyond),6ndmKwWqMozN2tcZqzCX4K,"[Fort Minor, Styles Of Beyond]","[7dWYWUbO68rXJOcyA7SpJk, 5bf6yYgHODBW5EreBZshpX]","Gym Motivation : Workout Motivation, Training ...","[rap rock, cali rap, rap rock]",0.688,0.8350,8,...,1,0.0911,0.05830,0.000003,0.0795,0.8800,84.858,230493,4,1
3,5e7e4fa1b96e22bd0ee61d9c,In Da Club,4RY96Asd9IefaL3X4LOLZ8,[50 Cent],[3q7HBObVc0L8jNeTe5Gofh],"Gym Motivation : Workout Motivation, Training ...","[east coast hip hop, gangster rap, hip hop, po...",0.902,0.7200,6,...,0,0.3470,0.26000,0.000000,0.0749,0.8050,90.059,193467,4,1
4,5e7e4fa1b96e22bd0ee61d9d,Stronger,4fzsfWzRhPawzqhX8Qt9F3,[Kanye West],[5K4W6rqBFWDnAN6FQUkS6x],"Gym Motivation : Workout Motivation, Training ...","[chicago rap, rap]",0.617,0.7170,10,...,0,0.1530,0.00564,0.000000,0.4080,0.4900,103.992,311867,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12678,5e7e63f38aa9abc22c18141a,Hallelujah,0uCGsNZqjHgiYO4BwN6Cjw,[Ben Laver],[3lTGvG2QAIoGMp7BKeH4C0],Calming Instrumental Covers,[],0.428,0.0415,7,...,0,0.0476,0.98500,0.942000,0.1060,0.2810,71.353,165882,4,0
12679,5e7e63f48aa9abc22c18141b,Close To Me,0kPLGFA5BjDGVkDbu4bLzE,[Mia Carrera],[00zoOzjXViIF4Fs8XQSK9m],Calming Instrumental Covers,"[calming instrumental, piano cover]",0.636,0.2410,4,...,1,0.0494,0.99000,0.902000,0.1510,0.1210,102.094,203587,4,0
12680,5e7e63f48aa9abc22c18141c,Love Me Like You Do,1nyxHNRjeyTlJQwsLMH7C0,[Anna Kitkowska],[45Miu8OyhYvkkQBXhn0MfQ],Calming Instrumental Covers,"[calming instrumental, chill guitar]",0.635,0.2310,11,...,0,0.0419,0.93900,0.947000,0.0925,0.3210,144.229,198208,4,0
12681,5e7e63f48aa9abc22c18141d,Fix You,2ZzqC4NSi374IufPYvGMP6,[Music Lab Collective],[1ylcY77FWeSVQKh5et1VGp],Calming Instrumental Covers,[classify],0.667,0.0498,3,...,1,0.0673,0.99100,0.890000,0.0818,0.0436,125.070,332427,4,0


Here is the dataframe we will use to train our classifier, next we define the features we want the classifier to learn and the feature we want to predict. These are named X and y respectively.

In [34]:
feature_cols = ['acousticness', 'danceability', 'duration_ms', 'energy', 'instrumentalness', 'key', 'liveness', 'mode', 'speechiness', 'tempo', 'time_signature']
X = dfTraining.loc[:, feature_cols]
y = dfTraining.workout_suitable

In [35]:
X

,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,mode,speechiness,tempo,time_signature
0,0.06220,0.548,297787,0.8470,0.000000,1,0.0816,1,0.1860,171.447,4
1,0.06030,0.687,205733,0.7930,0.000000,2,0.5820,1,0.1660,107.045,4
2,0.05830,0.688,230493,0.8350,0.000003,8,0.0795,1,0.0911,84.858,4
3,0.26000,0.902,193467,0.7200,0.000000,6,0.0749,0,0.3470,90.059,4
4,0.00564,0.617,311867,0.7170,0.000000,10,0.4080,0,0.1530,103.992,4
...,...,...,...,...,...,...,...,...,...,...,...
12678,0.98500,0.428,165882,0.0415,0.942000,7,0.1060,0,0.0476,71.353,4
12679,0.99000,0.636,203587,0.2410,0.902000,4,0.1510,1,0.0494,102.094,4
12680,0.93900,0.635,198208,0.2310,0.947000,11,0.0925,0,0.0419,144.229,4
12681,0.99100,0.667,332427,0.0498,0.890000,3,0.0818,1,0.0673,125.070,4


In [36]:
y

0        1
1        1
2        1
3        1
4        1
        ..
12678    0
12679    0
12680    0
12681    0
12682    0
Name: workout_suitable, Length: 12683, dtype: int64

## Prediction

#### Constructing a prediction model using X and y

In [38]:
rf = RandomForestClassifier(n_estimators=40)
workout_classifier = rf.fit(X, y)
workout_classifier

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=40,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Now we prepare the song recommendations to be tested 

In [39]:
X_predict = dfRecommendations.loc[:, feature_cols]
X_predict.shape

(20, 11)

### Using the classifer to predict the recommendations (X_predict)

In [40]:
new_prediction_class = workout_classifier.predict(X_predict)
new_prediction_class

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0])

 This is the array of predictions, we can build a new dataframe to display this information more clearly

In [41]:
dfClassifiedRecommendations = pd.DataFrame({
    'Track Name': dfRecommendations.track,
    'Track_id': dfRecommendations.track_id,
    'Prediction': new_prediction_class
})
dfClassifiedRecommendations

,Track Name,Track_id,Prediction
0,White Light,4sfORtxIQnO42AJpNy8Gsq,0
1,Keep,3OjWW823M5rbDuIr9mB06d,0
2,Starwood Choker,2w5l9sDXytt4lu0ZYFSoIX,0
3,Back to the Start,4Pdnt4VOAv26b1ILvtAoRj,0
4,Regenerative Being,1Rz3x1bF09JeEj2cTWH2mg,0
5,Merry,04NxrMXtaX4zjdwN12SdCF,0
6,Melancholia II,2UZXQrEi0A8ySbDuyeuMt3,0
7,Tears Don't Fall,5pKCDm2fw4k6D6C5Rk646C,1
8,Fyrsta,1EEJBUOORu9dLZQSeQkZOP,0
9,Reflector,1kAD3OQeKWZoT3oNj5bKLx,0


Here we have a list of songs with the classifiers prediction of if it is suitable for working out to or not. 
Now all that is left to do is display only the songs predicted as suitable for our hypothetical user.

In [42]:
dict1 = dfClassifiedRecommendations.to_dict('records')
for track in dict1:
    print(track)

{'Track Name': 'White Light', 'Track_id': '4sfORtxIQnO42AJpNy8Gsq', 'Prediction': 0}
{'Track Name': 'Keep', 'Track_id': '3OjWW823M5rbDuIr9mB06d', 'Prediction': 0}
{'Track Name': 'Starwood Choker', 'Track_id': '2w5l9sDXytt4lu0ZYFSoIX', 'Prediction': 0}
{'Track Name': 'Back to the Start', 'Track_id': '4Pdnt4VOAv26b1ILvtAoRj', 'Prediction': 0}
{'Track Name': 'Regenerative Being', 'Track_id': '1Rz3x1bF09JeEj2cTWH2mg', 'Prediction': 0}
{'Track Name': 'Merry', 'Track_id': '04NxrMXtaX4zjdwN12SdCF', 'Prediction': 0}
{'Track Name': 'Melancholia II', 'Track_id': '2UZXQrEi0A8ySbDuyeuMt3', 'Prediction': 0}
{'Track Name': "Tears Don't Fall", 'Track_id': '5pKCDm2fw4k6D6C5Rk646C', 'Prediction': 1}
{'Track Name': 'Fyrsta', 'Track_id': '1EEJBUOORu9dLZQSeQkZOP', 'Prediction': 0}
{'Track Name': 'Reflector', 'Track_id': '1kAD3OQeKWZoT3oNj5bKLx', 'Prediction': 0}
{'Track Name': 'Prelude For Time Feelers', 'Track_id': '3OtYYnDKs5oMzRorc4jnpW', 'Prediction': 0}
{'Track Name': 'Hearts On Fire', 'Track_id': '4

In [43]:
dictRecommend = []
for track in dict1: 
    if(track['Prediction'] == 1):
        dictRecommend.append(track)
for track in dictRecommend:
    print(track)

{'Track Name': "Tears Don't Fall", 'Track_id': '5pKCDm2fw4k6D6C5Rk646C', 'Prediction': 1}
{'Track Name': 'Hearts On Fire', 'Track_id': '4EFtG09zBCVqItRg3aSTrJ', 'Prediction': 1}
{'Track Name': 'Pain', 'Track_id': '33daQo3S8j2NcJstEHXwhQ', 'Prediction': 1}


## The recommendations are:

In [44]:
dfResults = pd.DataFrame.from_dict(dictRecommend)
dfResults

,Track Name,Track_id,Prediction
0,Tears Don't Fall,5pKCDm2fw4k6D6C5Rk646C,1
1,Hearts On Fire,4EFtG09zBCVqItRg3aSTrJ,1
2,Pain,33daQo3S8j2NcJstEHXwhQ,1
